In [ ]:
!pip install -U scikit-learn

In [ ]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/movies/metadata.zip
!unzip gdrive/My\ Drive/movies/dataset.zip 
!unzip gdrive/My\ Drive/movies/croppedonce.zip 
!unzip gdrive/My\ Drive/movies/cropped.zip 

In [ ]:
#!unzip gdrive/My\ Drive/movieset/trainingsmall.zip 
#!unzip gdrive/My\ Drive/movieset/testingsmall.zip 

In [1]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import json
import re
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss

import sklearn.preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
import glob
import csv
import cv2
import random
from PIL import Image
from itertools import product

In [2]:
path1 = "./Movie_Poster_Metadata/groundtruth"
temp_path = "./Movie_Poster_Metadata/temp_groundtruth"
path2 = "./Movie_Poster_Metadata/updated_groundtruth"
cropped_dataset = "./Dataset_Cropped"
# Jay's gdrive paths:
#path1 = "/content/drive/MyDrive/Uppsala University/Study material/NCML/Movie_Poster_Metadata/groundtruth"
#temp_path = "./Movie_Poster_Metadata/temp_groundtruth"
#path2 = "/content/gdrive/MyDrive/Uppsala University/Study material/NCML/Movie_Poster_Metadata/updated_groundtruth"

# path1 = "./Movie_Poster_Metadata/groundtruth"
# temp_path = "./Movie_Poster_Metadata/temp_groundtruth"
# path2 = "./Movie_Poster_Metadata/updated_groundtruth"

### Reading the input file and creating a clean one
Note: only run once

In [ ]:
"""
dir_list = os.listdir(path1)
 
if not os.path.exists(temp_path):
  os.makedirs(temp_path)    

if not os.path.exists(path2):
  os.makedirs(path2)



for file_name in dir_list:
    
    with open(path1+'/'+file_name,'r',encoding='utf-16-le') as file1:

        temp_file = open(temp_path+'/'+file_name,'w',encoding='utf-8')

        for line in file1.readlines():

            line = line.replace("}\n","},\n")
            
            # reading all lines that begin with "  "_id""
            y = re.findall("^  \"_id\"", line)
            if not y:
                temp_file.write(line)

    file1.close()
    temp_file.close()
    """

In [ ]:
"""
dir_list = os.listdir(temp_path)
 
for file_name in dir_list:
    
    with open(temp_path+'/'+file_name,'r',encoding='utf-8') as temp_file:
    
        file2 = open(path2+'/'+file_name,'w',encoding='utf-8')

        lines = temp_file.readlines()
        lines = lines[1:-1]

        file2.write("[{")
        file2.writelines(lines)
        file2.write("}]")
        
    temp_file.close()
    file2.close()

shutil.rmtree(temp_path)  
"""

### Augmenting the data set
Note: only run once

To-Do: Balance data according to occurence of genres. Summarize genres with little data.

In [ ]:
"""
path3 = "./Movie_Poster_Dataset"

# Going through all jpg-files, they are chopped up into 100x100 chunks and saved into a new folder
for dirname in os.listdir(path3):
    for filename in os.listdir(path3 + "/" + dirname):
        name, ext = os.path.splitext(filename)
        if(ext == '.jpg'):
            image = Image.open(os.path.join(path3 + "/" + dirname, filename))
            width, height = image.size
            chopsize = 100
            for x0 in range(0, width, chopsize):
                for y0 in range(0, height, chopsize):
                    if(y0+chopsize <= height and x0+chopsize <= width):
                        box = (x0, y0, x0+chopsize, y0+chopsize)
                        image.crop(box).save('gdrive/MyDrive/movies/Movie_Poster_Dataset_Cropped/%s.x%03d.y%03d.jpg' % (filename.replace('.jpg',''), x0, y0))
"""

In [ ]:
"""
path3 = "'gdrive/MyDrive/movies/Movie_Poster_Dataset"

# Going through all jpg-files, they are chopped up into 100x100 chunks and saved into a new folder
for dirname in os.listdir(path3):
    for filename in os.listdir(path3 + "/" + dirname):
        name, ext = os.path.splitext(filename)
        if(ext == '.jpg'):
            image = Image.open(os.path.join(path3 + "/" + dirname, filename))
            box = (0, 0, 100, 100)
            image.crop(box).save('./Movie_Poster_Dataset_Cropped_Once/%s.jpg' % (filename))
"""

In [ ]:
"""

#to be used later to augment data of underrepresented genres (balance data)

print('Nr of movies in json: '+str(len(dicts)))
missing = []
for obj in dicts:
    genrelist = obj.get('Genre').split(',')
    fname = obj.get('imdbID') + '.jpg'
    if(path.exists(fname)):
        for genre in genrelist:
            #copy the file with name obj.key("imdbID") to each genre folder
            if(genre == 'N/A'):
                shutil.copy2(os.path.join('.', fname), './NotApplicable')
            elif(genre == 'Adult' || genre == 'Game-Show' || genre == 'News' || genre == 'Reality-TV' || genre == 'Talk-Show' || genre == 'Western'):
                shutil.copy2(os.path.join('.', fname), './Other')
            else:
                shutil.copy2(os.path.join('.', fname), './'+genre.lstrip())
    else:
        missing.append(fname)


print('Nr of missing IDs: '+str(len(missing)))
"""

### Function to append all the json objects into dataframe 

In [3]:
dir_list = os.listdir(path2)

movies_df = pd.DataFrame()

for file_name in dir_list:    

#     try:
    df = pd.read_json(path2+'/'+file_name,encoding='utf-8',orient='records')
    df = df[['imdbID','Director','Genre','imdbRating']]
    movies_df = pd.concat([movies_df,df], ignore_index=True)

#     except:
#         print(file_name)
        
# print(movies_df.dtypes)
# print(movies_df.head(20))
# print(movies_df.shape)

### Creating multi-hot encoded genre vectors

In [4]:
#remove duplicates and set imdbID as index
movies_df = movies_df.drop_duplicates(subset=["imdbID"], keep="last")
movies_df.set_index("imdbID", inplace=True)

In [5]:
mlb = MultiLabelBinarizer()
multihot_temp = mlb.fit_transform(movies_df["Genre"].dropna().str.split(", "))

In [6]:
#['Action' 'Adult' 'Adventure' 'Animation' 'Biography' 
#'Comedy' 'Crime' 'Documentary' 'Drama' 'Family' 
#'Fantasy' 'Game-Show' 'History' 'Horror' 'Music' 
#'Musical' 'Mystery' 'N/A' 'News' 'Reality-TV' 
#'Romance' 'Sci-Fi' 'Short' 'Sport' 'Talk-Show' 
#'Thriller' 'War' 'Western']

multihot = [0]*len(multihot_temp)

item = 0
for vec in multihot_temp:
    
    new_vec = np.empty([20])
    for i in range(20):
        new_vec[i] = 0
    
    index = 0
    for i in range(28):
        
        #If genre is set in multihot-encoded vector
        if vec[i] == 1:
            
            #Adult, Game-Show, News, Reality-TV, Short, Talk-Show, Western -> Other
            other_genres = [1, 11, 18, 19, 22, 24, 27]
            if i in other_genres:
                new_vec[19] = 1 
                
            #Put Musical in same category as Music
            if i == 15:
                new_vec[index-1] = 1
        
            #Skip N/A
            if i == 17:
                continue
            
            else:
                new_vec[index] = 1
                index += 1
                
        if vec[i] == 0:
            relevant_genres = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 20, 21, 23, 25]
            if i in relevant_genres:
                index += 1
                
    multihot[item] = new_vec
    item += 1
#print(multihot)

In [7]:
genres_df = pd.DataFrame({"multihot":[multihot_temp.astype(int)]}, index = movies_df.index)
movies_df = pd.concat([movies_df, genres_df], axis=1 )
print(mlb.classes_)
print(movies_df.head(10))

['Action' 'Adult' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'Game-Show' 'History' 'Horror'
 'Music' 'Musical' 'Mystery' 'N/A' 'News' 'Reality-TV' 'Romance' 'Sci-Fi'
 'Short' 'Sport' 'Talk-Show' 'Thriller' 'War' 'Western']
                                           Director  \
imdbID                                                
tt0080684                            Irvin Kershner   
tt0081562                            Sidney Poitier   
tt0080339  Jim Abrahams, David Zucker, Jerry Zucker   
tt0080377                            Buddy Van Horn   
tt0081375                              Howard Zieff   
tt0080549                             Michael Apted   
tt0081529                               Hal Needham   
tt0080453                            Randal Kleiser   
tt0080455                               John Landis   
tt0081283                            Robert Redford   

                                Genre imdbRating  \
imdbID      

In [8]:
#create a dictionary with multi-hot encoded vectors; index = imdbID
multihot_dict = {movies_df.index.tolist()[i] : multihot[i] for i in range(0, len(multihot))}
#print(multihot_dict)

### Adding the images to the dataframe
Note: not used

In [ ]:
flist=glob.glob('./Movie_Poster_Dataset/*/*.jpg')

imdb_id_arr = ["0" for a in range(len(flist))]
image_arr = ["0" for a in range(len(flist))]
index = 0

for filename in flist:
        
    imdb_id = filename[filename.index("tt"):filename.index(".jpg")]
        
    imdb_id_arr[index] = imdb_id
                
    img = np.array(cv2.imread(filename))
    img = np.swapaxes(img, 2,0)
    img = np.swapaxes(img, 2,1)
    
    image_arr[index] = img
    
    index +=1 
        
image_dict = {
    "imdbID": imdb_id_arr,
    "Poster": image_arr
}

images_df = pd.DataFrame.from_dict(image_dict)
images_df = images_df.drop_duplicates(subset=["imdbID"], keep="last")
images_df.set_index("imdbID", inplace=True)
movies_df = pd.concat([movies_df, images_df], axis=1)
print(movies_df.head(10))

## Hyper Parameters

In [9]:
#training controls
batch_size = 1
number_of_labels = 20
epochs = 10
training_size = 0.7
learning_rate = 0.5 #0.1 #0.01 #0.001
dropout = [0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.15]
# input image dimensions
img_rows, img_cols = 100, 100

### Passing the images through a convolutional network

In [10]:
# the data holders
x_test = []
x_train = []
y_test = []
y_train = []

#images need to have the same size!!
flist=glob.glob('./Dataset_Cropped/*.jpg')

length=int(len(flist)*training_size)
i = 0

#create lists with input data (images) and output data (multi-hot encoded genre vectors)
for filename in flist:
        
    imdb_id = filename[filename.index("tt"):filename.index(".x")]
      
    if imdb_id in multihot_dict:
        img = np.array(cv2.imread(filename))
        img = np.swapaxes(img, 2,0)
        img = np.swapaxes(img, 2,1)
        
        genre_arr = np.empty([20])
        
        for j in range(len(multihot_dict[imdb_id])):
            genre_arr[j] = multihot_dict[imdb_id][j]
    
        if(i<length):  
            x_train.append(img)
            y_train.append(genre_arr)
        else:
            x_test.append(img)
            y_test.append(genre_arr)
        
        i +=1 

In [11]:
print(len(x_train))
print(len(y_train))

print(len(x_train[0]))
print(len(y_train[0]))

print(len(x_test[0]))
print(len(y_test[0]))

37032
37032
3
20
3
20


In [12]:
#converting the data from lists to numpy arrays
x_train=np.asarray(x_train,dtype=float)
x_test=np.asarray(x_test,dtype=float)
y_train=np.asarray(y_train,dtype=float)
y_test=np.asarray(y_test,dtype=float)

#scaling down the RGB data
x_train /= 255
x_test /= 255

#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)




x_train shape: (37032, 3, 100, 100)
37032 train samples
15872 test samples


### DenseNet-121

In [13]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True) # TODO: Modify the fully connected layer of denseNet

for param in model.parameters():
    param.requires_grad = False 
    # Replace the last fully-connected layer
    # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model.fc.in_features # num_ftrs = 1024
# model.classifier = nn.Linear(num_ftrs, 28)
# model.fc = nn.Linear(num_ftrs, 28) 
# sigmoid = nn.Sigmoid(model.fc)
# sigmoid(model.fc)
#TODO: make a sigmoid activation func, decision threshold of 0.3

model.classifier = nn.Sequential(
    nn.Linear(1024, 20),
    # nn.Linear(1024, 512),
    # nn.Dropout(p=0.1),
    # nn.ReLU(),
    # nn.Linear(512, 28),
    nn.Sigmoid()
)


model.double() 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"model loaded on {device}")
model.to(device) #hopefully runs model on cuda core.
print(model)

Using cache found in C:\Users\carol/.cache\torch\hub\pytorch_vision_v0.10.0


model loaded on cpu
DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats

## Custom CNN

In [ ]:
#BUG: forward function doesnt work and the 
# layers are made in the order in which they are declared in __init__
class Network(nn.Module):
    def __init__(self, input_shape=(3, img_rows, img_cols)):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=2)
        self.conv1_drop = nn.Dropout2d(p=dropout[0])
        self.conv2 = nn.Conv2d(128, 64, kernel_size=2)
        self.conv2_drop = nn.Dropout2d(p=dropout[1])
        self.conv3 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv3_drop = nn.Dropout2d(p=dropout[2])
        self.conv4 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv4_drop = nn.Dropout2d(p=dropout[3])
        self.conv5 = nn.Conv2d(64, 32, kernel_size=2)
        self.conv5_drop = nn.Dropout2d(p=dropout[4])
        self.conv6 = nn.Conv2d(32, 16, kernel_size=2)
        self.conv6_drop = nn.Dropout2d(p=dropout[5])
        
        self.n_size = self._get_conv_output(input_shape)
        """
        self.fc1 = nn.Linear(n_size, 16)
        self.fc1_drop = nn.Dropout(p=dropout[6])
        self.fc2 = nn.Linear(16, 16)
        self.fc2_drop = nn.Dropout(p=dropout[7])
        self.fc3 = nn.Linear(16, 8)
        self.fc3_drop = nn.Dropout(p=dropout[8])
        self.fc4 = nn.Linear(8, 28)
        """
        self.fc1 = nn.Linear(self.n_size, 28)
        self.sigmoid = nn.Sigmoid()

        
    def _get_conv_output(self, shape):
        bs = 1
        input = Variable(torch.rand(bs, *shape))
        # output_feat = self._forward_features(input)
        output_feat = self.forward(input)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size
        
    def _forward_features(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = F.relu(F.max_pool2d(self.conv5_drop(self.conv5(x)), 2))
        #x = F.relu(F.max_pool2d(self.conv6_drop(self.conv6(x)), 2))
        return x
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = F.relu(F.max_pool2d(self.conv5_drop(self.conv5(x)), 2))
        #x = F.relu(F.max_pool2d(self.conv6_drop(self.conv6(x)), 2))
        x = x.view(x.size(0), -1)
        # x = self.sigmoid(x)
        # x = torch.sigmoid(self.fc1(x))

        #x = F.relu(self.fc1_drop(self.fc1(x)))
        #x = F.relu(self.fc2_drop(self.fc2(x)))
        #x = F.relu(self.fc3_drop(self.fc3(x)))
        #x = self.fc4(x)
        return x

# model = Network()
# model.double() 

NameError: ignored

## Loss function

In [14]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0) #TODO: check best params here

## Train and Test functions

In [15]:
# Function to save the model
def saveModel():
    path = "./myFirstModel.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval() #TODO: check model.train() which is used along model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(f"model testing on {device}")
        some_accuracy_measure = 0
        final_accuracy = 0
        for i, (images, labels) in enumerate(test_loader,0):
            sum = 0
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            outputs = model(images) 
            # print(outputs)
            for j, label in enumerate(labels):
              n = sum + label.sum().item()
              # get the top n values of outputs
              _, predicted = torch.topk(outputs[j], int(n)) #TODO: find best threshold and calc F1.
              
              correct_predictions = 0
              for _, k in enumerate(predicted):
                if(label[k.item()].item() == 1):
                  correct_predictions += 1
                some_accuracy_measure = correct_predictions/n
            final_accuracy += some_accuracy_measure  
            # the label with the highest energy will be our prediction
            #_, predicted = torch.max(outputs.data, 1)
            #print("Predicted: ", predicted)
            total += labels.size(0)
            
            #accuracy += (predicted == labels).sum().item()
    print("Total: ", total)
    # compute the accuracy over all test images
    #accuracy = (100 * accuracy / total)
    accuracy = final_accuracy * 100 /total
    print("Accuracy;: ", accuracy)
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            #labels = labels.unsqueeze_(0)
            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i % 1000 == 0:    
                # print every 1000 (twice per epoch) 
                #print('[%d, %5d] loss: %.3f' %
                #      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0.0

        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy() #BUG: why is this in for loop? 
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

## Functions to display sample output

In [16]:
# Function to show the images
def imageshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# Function to test the model with a batch of images and show the labels predictions
def testBatch():
    # get batch of images from the test DataLoader  
    images, labels = next(iter(test_loader))

    # show all images as one image grid
    imageshow(torchvision.utils.make_grid(images))
   
    # Show the real labels on the screen 
    print('Real labels: ', ' '.join('%5s' % classes[labels[j]] 
                               for j in range(batch_size)))
  
    # Let's see what if the model identifiers the  labels of those example
    outputs = model(images)
    
    # We got the probability for every 10 labels. The highest (max) probability should be correct label
    _, predicted = torch.max(outputs, 1)
    
    # Let's show the predicted labels on the screen to compare with the real ones
    print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] 
                              for j in range(batch_size)))

## Run Model

In [ ]:
# for epoch in range(0, epochs):

train(epochs)
    #test()

# accuracy = testAccuracy()
# print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))

# best_accuracy = 0
# # we want to save the model if the accuracy is the best
# if accuracy > best_accuracy:
#     saveModel()
#     best_accuracy = accuracy

#TODO: plot learning curve

The model will be running on cpu device
model testing on cpu
Total:  15872.0
Accuracy;:  19.094947076613234
For epoch 1 the test accuracy over the whole test set is 19 %


### Passing the images through object detection

In [ ]:
batch_size = 20
epochs = 2
training_size = 0.7
learning_rate = 0.001
img_rows, img_cols = 100, 100

In [ ]:
# the data holders
x_test_yolo = []
x_train_yolo = []
y_test_yolo = []
y_train_yolo = []

#images need to have the same size!!
flist=glob.glob('./Movie_Poster_Dataset_Cropped_Once/*.jpg')

#pretrained YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

length=int(len(flist)*training_size)
i = 0

#create lists with input data (object confidence vector) and output data (multi-hot encoded genre vectors)
for filename in flist:
        
    imdb_id = filename[filename.index("tt"):filename.index(".jpg")]
      
    if imdb_id in multihot_dict:
        img = np.array(cv2.imread(filename))
        img = np.swapaxes(img, 2,0)
        img = np.swapaxes(img, 2,1)
        
        results = yolo_model(img, size = 100)
            
        #create an array for the 91 object categories and set initial confidence to 0
        obj_arr = np.empty([91])
        for x in range(91):
            obj_arr[x] = 0.0

        #update the confidence values according to the object detection results
        for obj in results.pandas().xyxy[0]:
            index =  results.pandas().xyxy[0]['class']
            obj_arr[index] = obj_arr[index] + results.pandas().xyxy[0].confidence
        
        #create multi-hot encoded genre vector
        genre_arr = np.empty([28])

        for j in range(len(multihot_dict[imdb_id])):
            genre_arr[j] = multihot_dict[imdb_id][j]
        
        if(i<length):                   
            x_train_yolo.append(obj_arr)
            y_train_yolo.append(genre_arr)
        else:
            x_test_yolo.append(obj_arr)
            y_test_yolo.append(genre_arr)
        
        i +=1 

In [ ]:
#converting the data from lists to numpy arrays
x_train_yolo=np.asarray(x_train_yolo,dtype=float)
x_test_yolo=np.asarray(x_test_yolo,dtype=float)
y_train_yolo=np.asarray(y_train_yolo,dtype=float)
y_test_yolo=np.asarray(y_test_yolo,dtype=float)

#printing stats about the features
print('x_train shape:', x_train_yolo.shape)
print(x_train_yolo.shape[0], 'train samples')
print(x_test_yolo.shape[0], 'test samples')

train_length = x_train_yolo.shape[0]

x_train_yolo=torch.from_numpy(x_train_yolo)
x_test_yolo=torch.from_numpy(x_test_yolo)
y_train_yolo=torch.from_numpy(y_train_yolo)
y_test_yolo=torch.from_numpy(y_test_yolo)

train = data_utils.TensorDataset(x_train_yolo, y_train_yolo)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test_yolo, y_test_yolo)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [ ]:
#fully connected layer after object detection
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #fully connected layer
        self.fc1 = nn.Linear(91, 28)
        
    def forward(self, x):
        x = self.fc1(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()

result = model.train()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate,
            alpha=0.9, eps=1e-08, weight_decay=0.0)
